In [1]:
# %env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from trl import RewardTrainer, RewardConfig
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW

# Level 1

## Reward Model

Загрузка модели и данных

In [3]:
sft_model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(sft_model_name)
sft_model = AutoModelForCausalLM.from_pretrained(sft_model_name)

In [4]:
dataset = load_dataset("juyoungml/HelpSteer2-binarized")

In [5]:
train_data = dataset['train']
val_data = dataset['validation']

Приведение датасета к implicit формату

In [6]:
def to_conversational_format(example):
    return {
        "chosen": [
            {"role": "user", "content": example["prompt"]},
            {"role": "assistant", "content": example["chosen"]},
        ],
        "rejected": [
            {"role": "user", "content": example["prompt"]},
            {"role": "assistant", "content": example["rejected"]},
        ],
    }

train_data = train_data.map(to_conversational_format)
val_data = val_data.map(to_conversational_format)

train_data = train_data.remove_columns(["prompt", "chosen_score", "rejected_score", "chosen_rationale", "rejected_rationale", "difficulty", "score_diff"])
val_data = val_data.remove_columns(["prompt", "chosen_score", "rejected_score", "chosen_rationale", "rejected_rationale", "difficulty", "score_diff"])

Обучение reward модели

In [14]:
rm_model = AutoModelForSequenceClassification.from_pretrained(sft_model_name, num_labels=1)
training_args = RewardConfig(
    output_dir="./reward_model_no_margin",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    logging_steps=20,
    gradient_checkpointing=True,  # reduce memory usage but train ~30% slower
    gradient_checkpointing_kwargs={"use_reentrant": False},
    gradient_accumulation_steps=1,
    learning_rate=5e-5,
    fp16=True,
    max_length=4096 * 2,  # Сюда влезет почти все
)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at HuggingFaceTB/SmolLM2-135M-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
trainer = RewardTrainer(
    model=rm_model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    processing_class=tokenizer,
)

Map:   0%|          | 0/7224 [00:00<?, ? examples/s]

Map:   0%|          | 0/7224 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7224 [00:00<?, ? examples/s]

In [9]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
20,0.750700
40,0.703400
60,0.692300
80,0.664400
100,0.724800
120,0.661200
140,0.675600
160,0.627800
180,0.612800
200,0.604600


TrainOutput(global_step=903, training_loss=0.6300229396534917, metrics={'train_runtime': 1192.7671, 'train_samples_per_second': 6.057, 'train_steps_per_second': 0.757, 'total_flos': 0.0, 'train_loss': 0.6300229396534917, 'epoch': 1.0})

Измерение количества пар $r_w > r_l$

In [10]:
rm_model = AutoModelForSequenceClassification.from_pretrained('./reward_model_no_margin/checkpoint-903', num_labels=1).cuda()

In [11]:
rm_model.eval()

correct = 0
for sample in tqdm(val_data):
    chosen = tokenizer.apply_chat_template(sample["chosen"], tokenize=False)
    rejected = tokenizer.apply_chat_template(sample["rejected"], tokenize=False)
    
    inputs = tokenizer(
        [chosen, rejected],
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=4096 * 2
    ).to("cuda")
    
    
    with torch.no_grad():
        outputs = rm_model(**inputs)
        scores = outputs.logits.squeeze().tolist()

    if scores[0] > scores[1]:
        correct += 1

  0%|          | 0/373 [00:00<?, ?it/s]

In [12]:
correct / len(val_data)

0.6648793565683646

Результат обучения на метрике так себе (но все еще лучше, чем 0.5). Я думаю, что нужно обучать дольше и/или с другими гиперпараметрами, но они зафиксированы в задании, поэтому дальше работа будет проводится с этой моделью.

Так как в датасете доступны скоры ответов, можно использовать и их для обучения
https://huggingface.co/docs/trl/main/en/reward_trainer#adding-a-margin-to-the-loss

В результате получилась reward модель немного хуже по метрике количества $r_w > r_l$ из валидационной выборки (код см ниже)

## Reinforce

Загрузка данных и модели

In [2]:
sft_model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
policy_model = AutoModelForCausalLM.from_pretrained(sft_model_name).cuda()
rm_model = AutoModelForSequenceClassification.from_pretrained('reward_model_no_margin/checkpoint-903').cuda()
tokenizer = AutoTokenizer.from_pretrained(sft_model_name)

policy_model.train()
rm_model.eval()

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576, padding_idx=2)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((576,), eps=1e-05)
  

In [3]:
MAX_LENGTH = 512  # Здесь уже меньше, чем в reward модели для экономии памяти.
# Однако в reward модель будем подавать необрезанные prompt и сгенерированный ответ

def preprocess_function(examples):
    return tokenizer(
        examples["prompt"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH,
        padding_side='left',
    )

In [4]:
dataset = load_dataset("juyoungml/HelpSteer2-binarized", split="train")
eval_dataset = load_dataset("juyoungml/HelpSteer2-binarized", split="validation")

dataset = dataset.map(preprocess_function)
eval_dataset = eval_dataset.map(preprocess_function)

In [5]:
bs = 6
max_gen_len = 128
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "prompt"])
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "prompt"])
loader = DataLoader(dataset, batch_size=bs, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=bs, shuffle=True)

Скользящее среднее в качестве бейзлайна

In [6]:
class MovingAverage:
    def __init__(self):
        self._total_reward = 0
        self._count = 0

    @property
    def average(self):
        return self._total_reward / self._count

    def __call__(self, rewards):
        for i in range(len(rewards)):  # Итерируемся по каждому реворду из батча по отдельности
            self._total_reward += rewards[i]
            self._count += 1
            rewards[i] -= self.average
        return rewards

Функция подсчета награды на валидационной выборке

In [17]:
def eval_reward(policy_model):
    with torch.no_grad():
        total_reward = 0
        for batch in tqdm(eval_loader):
            input_ids = batch["input_ids"].to('cuda')
            attention_mask = batch["attention_mask"].to('cuda')
            outputs = policy_model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_length=input_ids.size(1) + max_gen_len,
                    do_sample=True,
                    top_k=50,
                    return_dict_in_generate=True,
                )
            
            sequences = outputs.sequences
            gen_sequences = sequences[:, input_ids.size(1):]
        
            gen_texts = tokenizer.batch_decode(gen_sequences, skip_special_tokens=True)
            texts = []
            for i in range(len(batch['prompt'])):
                texts.append([
                    {"role": "user", "content": batch['prompt'][i]},
                    {"role": "assistant", "content": gen_texts[i]}
                ])
            chat = tokenizer.apply_chat_template(texts, tokenize=False)  # Приведение в conversational формат, на котором обучалась rm
            enc_rm = tokenizer(
                chat,
                return_tensors="pt",
                truncation=True,
                padding=True,
                max_length=4096,  # Здесь можно использовать длину больше, чем для промпта sft модели
            ).to("cuda")
            
            total_reward += rm_model(**enc_rm).logits.squeeze(-1).mean().item()
    return total_reward

In [16]:
max_gen_len = 128
n_iter = 0
cum_loss = 0

In [ ]:
optimizer = AdamW(policy_model.parameters(), lr=1e-5)  # lr=1e-5 зафиксирован для любых дообучений sft модели
rewards_history = []
mov_avg = MovingAverage()

for epoch in range(3):
    for batch in tqdm(loader):
        input_ids = batch["input_ids"].to('cuda')
        attention_mask = batch["attention_mask"].to('cuda')
        outputs = policy_model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=input_ids.size(1) + max_gen_len,
                do_sample=True,  # Здесь присутствует некоторая случайность, но это должно способствовать обучению лучше, чем argmax
                top_k=50,
                return_dict_in_generate=True,
            )
        
        sequences = outputs.sequences
        gen_sequences = sequences[:, input_ids.size(1):]

        # Подсчет log(pi(y|x))
        gen_logits = policy_model(sequences).logits[:, input_ids.size(1) - 1:-1, :]  # Логиты только сгенерированных токенов
        gen_log_probs = F.log_softmax(gen_logits, dim=-1)
        gen_log_probs = torch.gather(gen_log_probs, 2, gen_sequences.unsqueeze(-1)).squeeze(-1)  # Лог оценок вероятностей сгенерированных токенов
        gen_pad_mask = (gen_sequences != tokenizer.pad_token_id).float()
        # log(p(X)) = log(p(x_1) * p(x_2|x_1) * p(x_3|x_1, x_2) * ...) = log(p(x_1)) + log(p(x_2)) + ...
        gen_log_probs = (gen_log_probs * gen_pad_mask).sum(dim=1)  # Вроятность pad токенов не входит в оценку вероятности сгенерированного текста
    
        gen_texts = tokenizer.batch_decode(gen_sequences, skip_special_tokens=True)
        texts = []
        for i in range(len(batch['prompt'])):
            texts.append([
                {"role": "user", "content": batch['prompt'][i]},
                {"role": "assistant", "content": gen_texts[i]}
            ])
        chat = tokenizer.apply_chat_template(texts, tokenize=False)  # Приведение в conversational формат, на котором обучалась rm
        enc_rm = tokenizer(
            chat,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=4096,  # Здесь можно использовать длину больше, чем для промпта sft модели
        ).to("cuda")
        
        with torch.no_grad():
            rewards = rm_model(**enc_rm).logits.squeeze(-1)
    
        loss = -(mov_avg(rewards) * gen_log_probs).mean()  # loss из статьи
        
        if n_iter % 20 == 0 and n_iter > 0:
            print(f'n_iter: {n_iter}, loss: {cum_loss / 20}')
            cum_loss = 0
        if n_iter % 200 == 0:
            policy_model.eval()
            r = eval_reward(policy_model)  # Каждые 200 итераций смотрим на reward валидационной выборки
            rewards_history.append(r)
            print(f'\neval_reward: {r}')
            print()
            policy_model.train()

        cum_loss += loss.item()
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        n_iter += 1

  0%|          | 0/1204 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]


eval_reward: -287.66557264328003



In [9]:
rewards_history  # История ревордов каждые 200 итераций

[-287.66557264328003,
 -272.61764550209045,
 -253.94394040107727,
 -240.6589252948761,
 -226.70869159698486,
 -226.89586567878723,
 -220.69964003562927,
 -206.26513159275055,
 -216.38121724128723,
 -208.6601858139038,
 -200.4552869796753,
 -200.17305254936218,
 -204.647110581398,
 -195.3173370361328,
 -190.11309719085693,
 -196.9596209526062,
 -199.98877155780792]

In [14]:
n_iter  # Всего итераций для bs=6

3233

Финальный reward

In [11]:
policy_model.eval()
r = eval_reward(policy_model)
print(r)
policy_model.train()

  0%|          | 0/63 [00:00<?, ?it/s]

-193.72901272773743


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576, padding_idx=2)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((576,), eps=1e-05)
    (rotary_emb)

Реворд не дообученной sft модели

In [15]:
original_model = AutoModelForCausalLM.from_pretrained(sft_model_name).cuda()
original_model.eval()
print(eval_reward(original_model))
original_model.train()

  0%|          | 0/63 [00:00<?, ?it/s]

-290.31163215637207


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576, padding_idx=2)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((576,), eps=1e-05)
    (rotary_emb)

In [10]:
policy_model.save_pretrained('./policy_model_2')

## Доп: reward model с margin

In [ ]:
sft_model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(sft_model_name)
rm_model = AutoModelForSequenceClassification.from_pretrained(sft_model_name, num_labels=1)

In [ ]:
dataset = load_dataset("juyoungml/HelpSteer2-binarized")
train_data = dataset['train']
val_data = dataset['validation']

In [ ]:
def to_conversational_format(example):
    return {
        "chosen": [
            {"role": "user", "content": example["prompt"]},
            {"role": "assistant", "content": example["chosen"]},
        ],
        "rejected": [
            {"role": "user", "content": example["prompt"]},
            {"role": "assistant", "content": example["rejected"]},
        ],
    }

train_data = train_data.map(to_conversational_format)
val_data = val_data.map(to_conversational_format)

train_data = train_data.remove_columns(["prompt", "chosen_score", "rejected_score", "chosen_rationale", "rejected_rationale", "difficulty"])
val_data = val_data.remove_columns(["prompt", "chosen_score", "rejected_score", "chosen_rationale", "rejected_rationale", "difficulty"])

In [ ]:
train_data = train_data.rename_column("score_diff", "margin")  # Чтобы trl сам включил в лосс
val_data = val_data.rename_column("score_diff", "margin")

In [ ]:
training_args = RewardConfig(
    output_dir="./reward_model",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    logging_steps=20,
    gradient_checkpointing=True,  # reduce memory usage but train ~30% slower
    gradient_checkpointing_kwargs={"use_reentrant": False},
    gradient_accumulation_steps=1,
    learning_rate=5e-5,
    fp16=True,
    max_length=4096 * 2,  # Сюда влезет почти все
)

In [ ]:
trainer = RewardTrainer(
    model=rm_model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    processing_class=tokenizer,
)
trainer.train()

In [ ]:
import torch
from tqdm.notebook import tqdm

rm_model.eval()

correct = 0
for sample in tqdm(val_data):
    chosen = tokenizer.apply_chat_template(sample["chosen"], tokenize=False)
    rejected = tokenizer.apply_chat_template(sample["rejected"], tokenize=False)
    
    inputs = tokenizer(
        [chosen, rejected],
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=4096 * 2
    ).to("cuda")
    
    
    with torch.no_grad():
        outputs = rm_model(**inputs)
        scores = outputs.logits.squeeze().tolist()

    if scores[0] > scores[1]:
        correct += 1

In [13]:
correct / len(val_data)

0.6407506702412868


## Отчет Level 1

- Обучена reward модель с посредственной метрикой (0.66) количества доли пар угаданных оценок ответов $r_w > r_l$ с заданными гиперпараметрами и 1 эпохой. Предположительно, результат не очень из-за недообучения
- Добавление margin в loss reward модели не дал значимых изменений в метрике. Думаю, что также из-за недообучения
- С использованием reward модели реализован алгоритм reinforce с moving average baseline, который дал значительный прирост в средней награде на дообученной на нем sft моделе
- Итак, удалось добится reward=-193 на валидационной выборке после 3 эпох обучения sft модели по сравнению с не дообученной sft моделью с reward=-290. Средняя награда на отложенной выборке определенно значительно выросла. Это закономерный результат
- Стоит учесть, что, несмотря на то, что награда значительно выросла, реальное качество ответов может не улучшится или даже ухудшится из-за посредственной reward модели, как было замечено раннее

# Level 2

## Reward model

Загрузка моделей и данных, аналогично level 1

In [4]:
sft_model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(sft_model_name)
rm_model = AutoModelForSequenceClassification.from_pretrained(
    sft_model_name,
    num_labels=10,  # Модель возвращает логиты для 10 классов (оценок)
    problem_type="single_label_classification",  
)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at HuggingFaceTB/SmolLM2-135M-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
dataset = load_dataset("juyoungml/HelpSteer2-binarized")

In [6]:
def to_conversational_format(example):
    return {
        "chosen": [
            {"role": "user", "content": example["prompt"]},
            {"role": "assistant", "content": example["chosen"]},
        ],
        "rejected":[
            {"role": "user", "content": example["prompt"]},
            {"role": "assistant", "content": example["rejected"]},
        ],
    }

In [7]:
train_data = dataset["train"].map(to_conversational_format).remove_columns(
    ["prompt","chosen_score","rejected_score","chosen_rationale","rejected_rationale","difficulty","score_diff"]
)
val_data = dataset["validation"].map(to_conversational_format).remove_columns(
    ["prompt","chosen_score","rejected_score","chosen_rationale","rejected_rationale","difficulty","score_diff"]
)

В обычной rlhf используется $loss = -log(\sigma(r_w - r_l))$

Мы хотим максимизировать вероятность $p(y_w \succ y_l | x)$, имея $p_w(i)$ и $p_l(i)$, $i = 1...10$ - вероятность получить оценку i

Для этого можно применить $loss = -log \sum_{i=2}^{10}\sum_{j=1}^{i-1}p_w(i)*p_l(j)$ - просто явно посчитали $p(y_w > y_l)$ и навесили log, как в оригинальном лоссе

У этого лосса есть проблема, что он не учитывает, насколько далеки между собой i и j. То есть $p_w = (1, 0, ..., 0)$, $p_l = (0, ..., 0, 1)$ будет так же плохо, как $p_w = (1, 0, ..., 0)$, $p_l = (0, 1, 0 ..., 0)$

Чтобы учитывать это, можно применить что-то вроде аналога margin. Домножить каждое слагаемое на $\sigma(i - j)$.

Реализуем обучение reward модели с $loss = -log \sum_{i=1}^{10}\sum_{j=1}^{10}p_w(i)*p_l(j)*\sigma(i - j)$.

Также это должно сгладить функцию ошибки, что по идее должно способствовать обучению.

Можно также добавить гиперпараметр $\alpha$

$loss = -log \sum_{i=1}^{10}\sum_{j=1}^{10}p_w(i)*p_l(j)*\sigma(\alpha * (i - j))$

Для эксперимента возьму небольшое alpha, однако это уже позволит избежать случая, описанного выше

In [8]:
ALPHA = 0.2

class CustomRewardTrainer(RewardTrainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        if return_outputs or num_items_in_batch is not None:
            raise RuntimeError('return_outputs=True or num_items_in_batch not None')

        # получаем logits
        logits_w = model(input_ids=inputs['input_ids_chosen'], attention_mask=inputs['attention_mask_chosen']).logits
        logits_l = model(input_ids=inputs['input_ids_rejected'], attention_mask=inputs['attention_mask_rejected']).logits

        # Оценки вероятностей
        p_w = F.softmax(logits_w, dim=-1)
        p_l = F.softmax(logits_l, dim=-1)

        # Строим матрицу σ(i-j)
        device = logits_w.device
        idx = torch.arange(p_w.size(-1), device=device)
        i, j = idx[:, None], idx[None, :]
        sigma = torch.sigmoid(ALPHA * (i - j)).to(device)

        # Подсчет loss
        pref = (p_w.unsqueeze(2) * p_l.unsqueeze(1) * sigma).sum(dim=(1, 2))
        loss = -torch.log(pref + 1e-8).mean()  # Добавление 1e-8, чтобы избежать здесь ошибки

        return loss

Те же гиперпараметры

In [9]:
training_args = RewardConfig(
    output_dir="./reward_model_with_dist",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    logging_steps=20,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    gradient_accumulation_steps=1,
    learning_rate=5e-5,
    fp16=True,
    max_length=4096 * 2,
)

In [10]:
trainer = CustomRewardTrainer(
    model=rm_model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    processing_class=tokenizer,
)

trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
20,0.692200
40,0.685000
60,0.694500
80,0.693100
100,0.693100
120,0.693100
140,0.688800
160,0.687000
180,0.693000
200,0.677700


TrainOutput(global_step=903, training_loss=0.6836542161728193, metrics={'train_runtime': 1188.8209, 'train_samples_per_second': 6.077, 'train_steps_per_second': 0.76, 'total_flos': 0.0, 'train_loss': 0.6836542161728193, 'epoch': 1.0})

Посмотрим на ту же метрику качества, что и в level 1. Для это придется посчитать мат ожидание предсказанных оценок и сравнить

In [10]:
rm_model.eval()

correct = 0
for sample in tqdm(val_data):
    chosen = tokenizer.apply_chat_template(sample["chosen"], tokenize=False)
    rejected = tokenizer.apply_chat_template(sample["rejected"], tokenize=False)
    
    inputs = tokenizer(
        [chosen, rejected],
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=4096 * 2
    ).to("cuda")
    
    
    with torch.no_grad():
        outputs = rm_model(**inputs)
        scores = outputs.logits.squeeze()
        p = F.softmax(scores, dim=-1)
        ratings = torch.arange(1, 11, device=p.device).float()
        exp_r = (p * ratings).sum(dim=1)

    if exp_r[0] > exp_r[1]:
        correct += 1

  0%|          | 0/373 [00:00<?, ?it/s]

In [11]:
correct / len(val_data)

0.613941018766756

Качество по этой метрике получилось совсем немного хуже level 1, но зато теперь мы имеем целое распределение над оценками, а не просто скаляр

## Reinforce

Аналогично level 1

In [3]:
sft_model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
policy_model = AutoModelForCausalLM.from_pretrained(sft_model_name).cuda()
rm_model = AutoModelForSequenceClassification.from_pretrained('reward_model_with_dist/checkpoint-903').cuda()
tokenizer = AutoTokenizer.from_pretrained(sft_model_name)

policy_model.train()
rm_model.eval()

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576, padding_idx=2)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((576,), eps=1e-05)
  

In [5]:
MAX_LENGTH = 512

def preprocess_function(examples):
    return tokenizer(
        examples["prompt"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH,
        padding_side='left',
    )

In [6]:
dataset = load_dataset("juyoungml/HelpSteer2-binarized", split="train")
eval_dataset = load_dataset("juyoungml/HelpSteer2-binarized", split="validation")

dataset = dataset.map(preprocess_function)
eval_dataset = eval_dataset.map(preprocess_function)

In [7]:
bs = 6
max_gen_len = 128
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "prompt"])
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "prompt"])
loader = DataLoader(dataset, batch_size=bs, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=bs, shuffle=True)

In [8]:
class MovingAverage:
    def __init__(self):
        self._total_reward = 0
        self._count = 0

    @property
    def average(self):
        return self._total_reward / self._count

    def __call__(self, rewards):
        for i in range(len(rewards)):
            self._total_reward += rewards[i]
            self._count += 1
            rewards[i] -= self.average
        return rewards

In [8]:
mov_avg = MovingAverage()

Так как теперь у нас есть информация обо всем распределении, можно интегрировать ее в алгоритм reinforce, используя дополнительную информацию.

Понятно, что можно посчитать мат ожидание по распределению оценок и использовать его вместо скаляра из level 1

Помимо этого, также будем немного поощрять модель за сохранение энтропии в распределении. Это будет останавливать ее скатитываться в окрас ответов на хорошие и плохие, тем более что reward модель у нас показывает не очень хорошее качество. Ответы будут более разнообразными. Мы не могли бы применить такое, не имея распределения над оценками в rm

Также добавим еще регуляризационный коэффициент для энтропии = 0.1

В остальном, все аналогично level 1, но eval_reward здесь уже возвращает среднюю награду (просто нюанс)

In [9]:
def eval_reward_dist(policy_model):
    policy_model.eval()
    total_reward = 0.0
    n = 0
    with torch.no_grad():
        for batch in tqdm(eval_loader, desc="Eval"):
            input_ids = batch["input_ids"].cuda()
            attention_mask = batch["attention_mask"].cuda()
            outputs = policy_model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=input_ids.size(1) + max_gen_len,
                do_sample=True,
                top_k=50,
                return_dict_in_generate=True,
            )
            gen_sequences = outputs.sequences[:, input_ids.size(1):]
            gen_texts = tokenizer.batch_decode(gen_sequences, skip_special_tokens=True)
            chats = [
                [
                    {"role": "user", "content": batch['prompt'][i]},
                    {"role": "assistant", "content": gen_texts[i]}
                ]
                for i in range(len(gen_texts))
            ]
            chat_str = tokenizer.apply_chat_template(chats, tokenize=False)
            enc = tokenizer(
                chat_str,
                return_tensors="pt",
                truncation=True,
                padding=True,
                max_length=4096,
            ).to("cuda")
            
            logits = rm_model(**enc).logits
            p = F.softmax(logits, dim=-1)
            ratings = torch.arange(1, 11, device=p.device).float()
            exp_r = (p * ratings).sum(dim=1)  # Мат ожидание распределения оценок
            # энтропия RM: H(p) = -sum p log p
            ent = (-p * torch.log(p + 1e-8)).sum(dim=1)
            total_reward += ((exp_r - entropy_coeff * ent).sum().item())  # Общий reward с entropy_coeff
            n += exp_r.size(0)
    policy_model.train()
    return total_reward / n

In [10]:
n_iter = 0
cum_loss = 0.0
rewards_history = []
entropy_coeff = 0.1

Столько же итераций, как в level 1

In [10]:
optimizer = AdamW(policy_model.parameters(), lr=1e-5)

for epoch in range(3):
    for batch in tqdm(loader, desc=f"Epoch {epoch+1}"):
        input_ids = batch["input_ids"].cuda()
        attention_mask = batch["attention_mask"].cuda()

        # Генерация
        outputs = policy_model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=input_ids.size(1) + max_gen_len,
            do_sample=True,
            top_k=50,
            return_dict_in_generate=True,
        )
        sequences = outputs.sequences
        gen_sequences = sequences[:, input_ids.size(1):]

        # Лог-пробы π
        gen_logits = policy_model(sequences).logits[:, input_ids.size(1)-1:-1, :]
        log_probs = F.log_softmax(gen_logits, dim=-1)
        log_probs = torch.gather(log_probs, 2, gen_sequences.unsqueeze(-1)).squeeze(-1)
        mask = (gen_sequences != tokenizer.pad_token_id).float()
        seq_logprob = (log_probs * mask).sum(dim=1)

        # RM ожидаемая оценка + энтропия
        gen_texts = tokenizer.batch_decode(gen_sequences, skip_special_tokens=True)
        chats = [
            [
                {"role": "user", "content": batch['prompt'][i]},
                {"role": "assistant", "content": gen_texts[i]}
            ]
            for i in range(len(gen_texts))
        ]
        chat_str = tokenizer.apply_chat_template(chats, tokenize=False)
        enc = tokenizer(
            chat_str,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=4096,
        ).to("cuda")
        with torch.no_grad():
            logits = rm_model(**enc).logits
            p = F.softmax(logits, dim=-1)
            ratings = torch.arange(1, 11, device=p.device).float()
            exp_r = (p * ratings).sum(dim=1)  # Мат ожидание распределения оценок
            ent = (-p * torch.log(p + 1e-8)).sum(dim=1)  # Энтропия
            rewards = exp_r - entropy_coeff * ent

        # REINFORCE loss с baseline
        adv = mov_avg(rewards)
        loss = -(adv * seq_logprob).mean()

        # Шаг оптимизации
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Логирование
        cum_loss += loss.item()
        if n_iter and n_iter % 20 == 0:
            print(f"Iter {n_iter:05d}  avg_loss={cum_loss/20:.4f}")
            cum_loss = 0.0
        if n_iter and n_iter % 200 == 0:
            avg_r = eval_reward_dist(policy_model)
            rewards_history.append(avg_r)
            print(f"\n>> eval_reward: {avg_r:.4f}\n")

        n_iter += 1

Epoch 1:   0%|          | 0/1204 [00:00<?, ?it/s]

Iter 00020  avg_loss=10.2487
Iter 00040  avg_loss=72.9807
Iter 00060  avg_loss=16.6656
Iter 00080  avg_loss=-12.3954
Iter 00100  avg_loss=107.6640
Iter 00120  avg_loss=30.4735
Iter 00140  avg_loss=106.6436
Iter 00160  avg_loss=114.3030
Iter 00200  avg_loss=101.7930


Eval:   0%|          | 0/63 [00:00<?, ?it/s]


>> eval_reward: 3.4773

Iter 00220  avg_loss=146.9995
Iter 00240  avg_loss=116.0670
Iter 00260  avg_loss=30.8886
Iter 00280  avg_loss=113.5096
Iter 00300  avg_loss=71.1413
Iter 00320  avg_loss=13.4653
Iter 00340  avg_loss=4.4304
Iter 00360  avg_loss=-62.0211
Iter 00380  avg_loss=9.9224
Iter 00400  avg_loss=-471.0392


Eval:   0%|          | 0/63 [00:00<?, ?it/s]

...

Eval:   0%|          | 0/63 [00:00<?, ?it/s]


>> eval_reward: 4.2147

Iter 00820  avg_loss=-1777.8043
Iter 00840  avg_loss=-1761.4774
Iter 00860  avg_loss=-750.6438
Iter 00880  avg_loss=-1979.2062
Iter 00900  avg_loss=-1984.1066
Iter 00920  avg_loss=195.9078
Iter 00940  avg_loss=-1521.1585
Iter 00960  avg_loss=-1936.7599
Iter 00980  avg_loss=-1071.8721
Iter 01000  avg_loss=-1556.7316


Eval:   0%|          | 0/63 [00:00<?, ?it/s]


>> eval_reward: 4.2917

Iter 01020  avg_loss=-1038.5451
Iter 01040  avg_loss=-1183.2266
Iter 01060  avg_loss=-1124.0564
Iter 01080  avg_loss=-2015.5051
Iter 01100  avg_loss=-1471.6347
Iter 01120  avg_loss=-1390.9599


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [12]:
rewards_history

[3.477266660005094,
 3.7778431212294836,
 4.118272331380972,
 4.214707447440631,
 4.2917454639005275,
 4.329766364902977,
 4.284382241021532,
 4.200179849169529,
 4.136497256583247,
 4.180881199184755,
 4.197088994545208,
 4.063511586381027,
 4.1756648793616815,
 4.166365514811498,
 4.242778195133158,
 4.158104339170072,
 3.991916802869086,
 4.075044806457397]

In [11]:
policy_model.save_pretrained('./policy_model_dist')

## Сравнение моделей

Так как модели обучены на абсолютно разных reward model, сравним reward на каждой из них

Сначала сравним на reward model из level 2

In [3]:
sft_model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(sft_model_name)
dist_policy_model = AutoModelForCausalLM.from_pretrained('./policy_model_dist').cuda()
untrained_model = AutoModelForCausalLM.from_pretrained(sft_model_name).cuda()
policy_model = AutoModelForCausalLM.from_pretrained('./policy_model_2').cuda()
rm_model = AutoModelForSequenceClassification.from_pretrained('reward_model_with_dist/checkpoint-903').cuda()

dist_policy_model.eval()
untrained_model.eval()
policy_model.eval()
rm_model.eval()

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576, padding_idx=2)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((576,), eps=1e-05)
  

Необученная

In [13]:
eval_reward_dist(untrained_model)

Eval:   0%|          | 0/63 [00:00<?, ?it/s]

3.24191510964974

Дообученная на level 2 rm

In [11]:
eval_reward_dist(dist_policy_model)

Eval:   0%|          | 0/63 [00:00<?, ?it/s]

4.032983042279772

Дообученная на level 1 rm

In [14]:
eval_reward_dist(policy_model)

Eval:   0%|          | 0/63 [00:00<?, ?it/s]

4.30431632637658

Теперь сравним на reward model из level 1

In [15]:
rm_model = AutoModelForSequenceClassification.from_pretrained('reward_model_no_margin/checkpoint-903').cuda()
rm_model.eval()

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576, padding_idx=2)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((576,), eps=1e-05)
  

In [18]:
eval_reward(untrained_model)

  0%|          | 0/63 [00:00<?, ?it/s]

-291.08098459243774

In [19]:
eval_reward(dist_policy_model)

  0%|          | 0/63 [00:00<?, ?it/s]

-295.22210478782654

In [21]:
eval_reward(policy_model)

  0%|          | 0/63 [00:00<?, ?it/s]

-208.5806030035019

## Отчет по level 2

- Обучена reward модель, предсказывающая распределение над оценками (как именно см выше) с метрикой правильно проранжированных пар, близкой к reward model из level 1
- Используя распределения из reward model level 2, дообучена sft модель (как именно см выше)
- Однако средняя награда на reward model level 2 оказалась немного выше у sft модели level 1. Скорее всего, это связано с недообученной reward моделью, или плохо подобранным loss для дообучения sft level 2. Возможно, стоит брать другие характеристики распределения (мат ожидание и что-то еще помимо энтропии).
- В награде на reward model level 1 также побеждает sft level 1 с большим отрывом. А sft level 2 показывает результат, близкий к недообученной модели. Я думаю, что причины те же, что и в предыдущем пункте. Здесь мог еще сыграть тот факт, что reward model level 2 сложнее, чем reward model level 1, а значит обучение у нее может быть более сложным/прихотливым. Обучали же мы их с заранее заданными параметрами

В целом, выглядит, что эксперимент по level 2 неудачный. Дальнейшие шаги, которые стоит попробовать

- Пробовать дообучить reward модели, чтобы они показывали лучшую метрику правильно проранжированных пар
- Включить другие характеристики распределения в loss модели sft level 2
- Пробовать разные гиперпараметры, в тч те, которые содержаться в loss'е и были введеные выше